In [1]:
# import os, sys
# import numpy
# import pandas
# from pandas import DataFrame, Series
# import argparse
# from multiprocessing import Pool, cpu_count

# import gc
# #import psutil
# # process = psutil.Process(os.getpid())

# import simons_meta_data
# import hg19_chrom_sizes

# #script_dir = os.path.dirname(os.path.realpath(__file__))
# sys.path.insert(0, '/home/kmt/simons/faststorage/people/kmt/notebooks')
# import analysis_globals


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--nr_wins', type=int, default=5, help='')
parser.add_argument('--offset', type=int, default=100000, help='')
parser.add_argument('--cpus', type=int, help='')
##### Added these two options
parser.add_argument('--min-sweep-clade-percent', dest='min_sweep_clade_percent', type=int)
parser.add_argument('--pwdist-cutoff', dest='pwdist_cutoff', type=float)                
# parser.add_argument('--dump-dist-twice', dest='dump_dist_twice', type=str, default=None, help='')
parser.add_argument('dist_file_name', type=str, help='')
parser.add_argument('sweep_data_file_name', type=str, help='')
args = parser.parse_args()

# class Args():
#     def __init__(self):
#         self.nr_wins = 5
#         self.offset = 100000
#         self.cpus = 1
#         self.min_sweep_clade_percent = 1
#         self.pwdist_cutoff = 5e-05
#         self.dist_file_name = '/home/kmt/simons/faststorage/people/kmt/steps/male_dist_admix_masked_stores/male_dist_data_chrX_100kb_twice.hdf'
#         self.sweep_data_file_name = '/home/kmt/simons/faststorage/people/kmt/steps/male_dist_admix_masked_stores/5e-05/sweep_data_5e-05_1%.hdf'

# args = Args()

if args.cpus:
    nr_cpu = args.cpus
else:
    nr_cpu = int(os.environ.get('SLURM_JOB_CPUS_PER_NODE'))

nr_wins = args.nr_wins
offset = args.offset
offsets = [x * offset for x in range(nr_wins)]
window_size = len(offsets) * offset

------

In [ ]:
import os, sys
import numpy
import pandas
from pandas import DataFrame, Series
import argparse
from multiprocessing import Pool, cpu_count

import gc

import simons_meta_data
import hg19_chrom_sizes

import networkx as nx
import itertools

script_dir = os.path.dirname(os.path.realpath(__file__))
sys.path.insert(0, script_dir + '/../notebooks')
import analysis_globals


def call_rolling_windows(df, pwdist_cutoff, min_sweep_clade_size):
    """

    """

    # build data frame to return
    cols = ['indiv_1', 'start', 'end', 'off']
    result_df = pandas.DataFrame(list(df.groupby(cols).groups.keys()), columns=cols)

    # Get mean dist between pairs in the 500kb window (make mean nan if any 100kb is nan):
    win_dist = df.groupby(['indiv_1', 'indiv_2']).dist.agg(numpy.mean)
    
    # Build graph
    graph = nx.Graph()
    for tup in win_dist[win_dist <= pwdist_cutoff].reset_index().itertuples():
        graph.add_edge(tup.indiv_1, tup.indiv_2)
        
    cliques = sorted(nx.algorithms.clique.find_cliques(graph), key=len)

    # find swept
    result_df['called'] = False
    result_df['mean_clade_dist'] = numpy.nan
    result_df['clade_size'] = numpy.nan
    
    for clique in cliques:

        if len(clique) >= min_sweep_clade_size:
            
            # get indiv pairs. (in just one orientation). that is fine because df has both orientations

            called = result_df.indiv_1.isin(clique)
            #result_df['called'] = called | result_df.called # do not make it False if it is already True   
            
            result_df.loc[called, 'called'] = True
            
            # clade_size is the size of the clique
            result_df.loc[called, 'clade_size'] = len(clique) 
            
            # mean_clade_dist is the mean dist of the clique
            indiv_pairs = list(itertools.combinations(sorted(clique), 2))
            result_df.loc[called, 'mean_clade_dist'] = win_dist.loc[indiv_pairs].mean()                
                
    if 'dist_af' in df.columns:
        
        # Get mean dist between pairs in the 500kb window (make mean nan if any 100kb is nan):
        win_dist = df.groupby(['indiv_1', 'indiv_2']).dist_af.agg(numpy.mean)

        # Build graph
        graph = nx.Graph()
        for tup in win_dist[win_dist <= pwdist_cutoff].reset_index().itertuples():
            graph.add_edge(tup.indiv_1, tup.indiv_2)

        cliques = sorted(nx.algorithms.clique.find_cliques(graph), key=len)

        # find swept
        result_df['called_af'] = False
        result_df['mean_clade_dist_af'] = numpy.nan
        result_df['clade_size_af'] = numpy.nan

        for clique in cliques:

            if len(clique) >= min_sweep_clade_size:
                # get indiv pairs. (in just one orientation). that is fine because df has both orientations

                called = result_df.indiv_1.isin(clique)

                result_df.loc[called, 'called_af'] = True

                # clade_size is the size of the clique
                result_df.loc[called, 'clade_size_af'] = len(clique) 

                # mean_clade_dist is the mean dist of the clique
                indiv_pairs = list(itertools.combinations(sorted(clique), 2))
                result_df.loc[called, 'mean_clade_dist_af'] = win_dist.loc[indiv_pairs].mean()

    return result_df


def call_swept(df):
    """
    Takes a df with all rolling window data for an indivisual for one 100kb window.
    Call each 100kb window as sweept if any overlapping rolling window is called as swept.
    Compute clade size and mean clade dist as from the rolling window with the largest clade size.
    """
    max_clade_size = df.clade_size.max()
    offset_with_largest_clique = (df.groupby('off')
                             .filter(lambda df: (df.clade_size == max_clade_size).all() and df.called.all())
                            )

    assert len(df) == len(df.off.unique()) # one row for each offset

    if 'called_af' in df.columns:
        max_clade_size_af = df.clade_size_af.max()
        offset_with_largest_clique_af = (df.groupby('off')
                                .filter(lambda df: (df.clade_size_af == max_clade_size_af).all() and df.called_af.all())
                                )

        return DataFrame(dict(called=[df.called.any()], 
                            clade_size=[max_clade_size],
                            clade_mean_dist=[offset_with_largest_clique.mean_clade_dist.mean()], # actually mean over identical numbers
                            called_af=[df.called_af.any()], 
                            clade_size_af=[max_clade_size_af],
                            clade_mean_dist_af=[offset_with_largest_clique_af.mean_clade_dist.mean()])) # actually mean over identical numbers
    else:
        return DataFrame(dict(called=[df.called.any()], 
                            clade_size=[max_clade_size],
                            clade_mean_dist=[offset_with_largest_clique.mean_clade_dist.mean()])) # actually mean over identical numbers
    


def _window_stats(df, pwdist_cutoff, min_sweep_clade_size):
    if 'dist_af' in df.columns:
        return pandas.DataFrame({
                                'mean_dist': [df.dist.mean()],
                                'mean_dist_to_africans': [df.loc[df.region_2 == 'Africa', 'dist'].mean()],
                                'mean_dist_af': [df.dist_af.mean()],
                                'mean_dist_to_africans_af': [df.loc[df.region_2 == 'Africa', 'dist_af'].mean()],
                                'win_swept': (df.dist <= pwdist_cutoff).sum() >= \
                                                min_sweep_clade_size,
                                'win_swept_af': (df.dist_af <= pwdist_cutoff).sum() >= 
                                                min_sweep_clade_size,
                                'prop_indivs_missing': [numpy.isnan(df.dist).sum() / df.dist.size]
                                })
    else:
        return pandas.DataFrame({
                        'mean_dist': [df.dist.mean()],
                        'win_swept': (df.dist <= pwdist_cutoff).sum() >= \
                                        min_sweep_clade_size,
                        'prop_indivs_missing': [numpy.isnan(df.dist).sum() / df.dist.size]
                        })

In [6]:
###################################################################

# parser = argparse.ArgumentParser()
# parser.add_argument('--nr_wins', type=int, default=5, help='')
# parser.add_argument('--offset', type=int, default=100000, help='')
# parser.add_argument('--cpus', type=int, help='')
# parser.add_argument('--min-sweep-clade-percent', dest='min_sweep_clade_percent', type=int)
# parser.add_argument('--pwdist-cutoff', dest='pwdist_cutoff', type=float)                
# parser.add_argument('dist_file_name', type=str, help='')
# parser.add_argument('sweep_data_file_name', type=str, help='')
# args = parser.parse_args()

# if args.cpus:
#     nr_cpu = args.cpus
# else:
#     nr_cpu = int(os.environ.get('SLURM_JOB_CPUS_PER_NODE'))

# nr_wins = args.nr_wins
# offset = args.offset
# offsets = [x * offset for x in range(nr_wins)]
# window_size = len(offsets) * offset

###################################################################

class Args():
    def __init__(self):
        self.nr_wins = 5
        self.offset = 100000
        self.cpus = 1
        self.min_sweep_clade_percent = 30
        self.pwdist_cutoff = 5e-05
        self.dist_file_name = '/home/kmt/simons/faststorage/people/kmt/steps/male_dist_admix_masked_stores/male_dist_data_chrX_100kb_twice.hdf'
        self.sweep_data_file_name = '/home/kmt/simons/faststorage/people/kmt/steps/male_dist_admix_masked_stores/5e-05/sweep_data_5e-05_1%.hdf'

args = Args()

if args.cpus:
    nr_cpu = args.cpus
else:
    nr_cpu = int(os.environ.get('SLURM_JOB_CPUS_PER_NODE'))

nr_wins = args.nr_wins
offset = args.offset
offsets = [x * offset for x in range(nr_wins)]
window_size = len(offsets) * offset
###################################################################


In [7]:
all_male_dist_twice = pandas.read_hdf(args.dist_file_name)

# remove unused columns
to_keep = ['indiv_1', 'indiv_2', 'start', 'end', 'dist', 'dist_af', 
           'region_1', 'region_2', 'pop_1', 'region_label_1', 'region_id_1']
to_drop = [x for x in all_male_dist_twice.columns if x not in to_keep]
all_male_dist_twice.drop(to_drop, axis=1, inplace=True)
gc.collect()

nr_indiv = all_male_dist_twice.indiv_1.unique().size

MIN_SWEEP_CLADE_SIZE = round(nr_indiv * args.min_sweep_clade_percent / 100)

# partial version of window_stats
import functools
window_stats = functools.partial(_window_stats, pwdist_cutoff=args.pwdist_cutoff, min_sweep_clade_size=MIN_SWEEP_CLADE_SIZE)

# merge window sweep info with distance data
if 'dist_af' in all_male_dist_twice.columns:
    # this is not a simulation
    gr_cols = ['indiv_1', 'start', 'end', 'pop_1', 'region_label_1', 'region_id_1', 'region_1']
else:
    gr_cols = ['indiv_1', 'start', 'end']
stats_data = (all_male_dist_twice
        .groupby(gr_cols)
        .apply(window_stats)
        .reset_index(level=gr_cols)
        )

lst = list()
# loop over five offsets of 500kb windows
for off in offsets:
    print(off)
    groups = (all_male_dist_twice
                .assign(off=off, # keep offset
                        roll_win = lambda df: (off + df.start) // window_size) # label for rolling 500kb window
                .groupby(['roll_win', 'off'], as_index=False)
              
              ### ADDED THIS TO ENSURE NO CALLS ARE MADE ON WINDOWS SMALLER THAN 500KB:
                .filter(lambda df: df.start.unique().size == nr_wins)             
                .groupby(['roll_win', 'off'], as_index=False)

                )
    # with Pool(nr_cpu) as p:
    #     df = pandas.concat(p.map(call_rolling_windows, [group for name, group in groups]))
    ##### added pwdist arg to function call

    lst.append(groups.apply(call_rolling_windows, args.pwdist_cutoff, MIN_SWEEP_CLADE_SIZE))

del all_male_dist_twice
gc.collect()

# concatenate data frames for each offset and call windows as swept
sweep_calls = (pandas.concat(lst)
                .groupby(['indiv_1', 'start', 'end'])
                .apply(call_swept)
                .reset_index(level=['indiv_1', 'start', 'end'])
                )

del lst[:]
gc.collect()

sweep_data = stats_data.merge(sweep_calls, on=['indiv_1', 'start', 'end'])

del stats_data
del sweep_calls
gc.collect()

# get run length of swept windows and call windows as part of sweeps (ECH haplotypes)
def run_id(sr):
    return (sr != sr.shift()).cumsum()

sweep_data['run_id'] = (sweep_data
                        .groupby('indiv_1')['called']
                        .apply(run_id)
                       )
sweep_data['run_length'] = (sweep_data
                            .groupby(['indiv_1', 'run_id'])['run_id']
                            .transform(numpy.size)
                           )
sweep_data['swept'] = numpy.bitwise_and(sweep_data['called'], 
                                        sweep_data['run_length'] >= analysis_globals.min_run_length)

if 'called_af' in sweep_data.columns:
    # this is not a simulation
    sweep_data['run_id_af'] = (sweep_data
                            .groupby('indiv_1')['called_af']
                            .apply(run_id)
                        )
    sweep_data['run_length_af'] = (sweep_data
                                .groupby(['indiv_1', 'run_id_af'])['run_id_af']
                                .transform(numpy.size)
                            )
    sweep_data['swept_af'] = numpy.bitwise_and(sweep_data['called_af'], 
                                            sweep_data['run_length_af'] >= analysis_globals.min_run_length)

gc.collect()

# write to hdf output file
sweep_data.to_hdf(args.sweep_data_file_name, 'df', mode='w', format='table')

-----

In [5]:
process.memory_info().rss / 1024**3

0.07740020751953125

In [6]:
###### no longer reading dist_file name                                    
# male_dist_data = pandas.read_hdf(args.dist_file_name)

# all_male_dist_twice = dist_twice(male_dist_data)
###### reading twice file instead.

###### NEW ##########
# all_male_dist_twice = pandas.read_hdf(args.dist_file_name)
all_male_dist_twice = pandas.read_hdf('test.hdf')

to_keep = ['indiv_1', 'indiv_2', 'start', 'end', 'dist', 'dist_af', 
           'region_1', 'region_2', 'pop_1', 'region_label_1', 'region_id_1']
to_drop = [x for x in all_male_dist_twice.columns if x not in to_keep]
all_male_dist_twice.drop(to_drop, axis=1, inplace=True)
###### NEW ##########

##### defining MIN_SWEEP_CLADE_SIZE using arg to script
nr_indiv = all_male_dist_twice.indiv_1.unique().size

MIN_SWEEP_CLADE_SIZE = round(nr_indiv * args.min_sweep_clade_percent / 100)

print('Loaded dist twice', process.memory_info().rss / 1024**3)

process.memory_info().rss / 1024**3

Loaded dist twice 3.3942184448242188


3.3942298889160156

In [7]:
process.memory_info().rss / 1024**3, all_male_dist_twice.memory_usage(deep=True).sum() / 1024 ** 3

(3.394367218017578, 1.2441185507923365)

In [14]:

#### NEW #################

## optimize data frame in script that generates dist twice frame..

def optimize_data_frame(df, down_int='integer'):
    # down_int can be 'unsigned'
    
    converted_df = pandas.DataFrame()

    floats_optim = (df
                    .select_dtypes(include=['float'])
                    .apply(pandas.to_numeric,downcast='float')
                   )
    converted_df[floats_optim.columns] = floats_optim

    ints_optim = (df
                    .select_dtypes(include=['int'])
                    .apply(pandas.to_numeric,downcast=down_int)
                   )
    converted_df[ints_optim.columns] = ints_optim

    for col in df.select_dtypes(include=['object']).columns:
        num_unique_values = len(df[col].unique())
        num_total_values = len(df[col])
        if num_unique_values / num_total_values < 0.5:
            converted_df[col] = df[col].astype('category')
        else:
            converted_df[col] = df[col]

    unchanged_cols = df.columns[~df.columns.isin(converted_df.columns)]
    converted_df[unchanged_cols] = df[unchanged_cols]

    # keep columns order
    converted_df = converted_df[df.columns]      
            
    return converted_df

#all_male_dist_twice = optimize_data_frame(all_male_dist_twice, down_int='unsigned')
all_male_dist_twice = optimize_data_frame(all_male_dist_twice, down_int='unsigned')
gc.collect()

#### NEW #################


2883

In [8]:
all_male_dist_twice.memory_usage(deep=True).sum() / 1024 ** 3, process.memory_info().rss / 1024**3

(1.2441185507923365, 3.394390106201172)

In [9]:
#all_male_dist_twice.to_hdf('test.hdf', 'df', format='table', mode='w')

In [10]:

# all_male_dist_twice = all_male_dist_twice.iloc[:100000]
###########################################

In [11]:
# off = 0

# groups = (all_male_dist_twice
#             .assign(off=off, # keep offset
#                     roll_win = lambda df: (off + df.start) // window_size) # label for rolling 500kb window
#             .groupby(['indiv_1', 'roll_win', 'off'], )
#             )

In [12]:
# group = groups.get_group(('B_Han-3', 200, 0))


In [13]:
# %%timeit
# x = numpy.empty(5)
# x[:] = numpy.nan

In [14]:
# %%timeit
# x = [numpy.nan] * 5

In [15]:
# %%timeit
# Series(numpy.nan, index=[1, 2, 3, 4, 5])

In [16]:
# %%timeit
# new_df = new_call_rolling_windows(group, args.pwdist_cutoff, MIN_SWEEP_CLADE_SIZE)

In [17]:
# %%timeit
# old_df = old_call_rolling_windows(group, args.pwdist_cutoff, MIN_SWEEP_CLADE_SIZE)

In [18]:
# old_df = old_call_rolling_windows(group, args.pwdist_cutoff, MIN_SWEEP_CLADE_SIZE)
# # 
# new_df.equals(old_df)

In [19]:
# %%time

# off = 0

# groups = (all_male_dist_twice
#             .assign(off=off, # keep offset
#                     roll_win = lambda df: (off + df.start) // window_size) # label for rolling 500kb window
#             .groupby(['indiv_1', 'roll_win', 'off'], as_index=False)
#             )

# new_df = groups.apply(new_call_rolling_windows, args.pwdist_cutoff, MIN_SWEEP_CLADE_SIZE)

In [20]:
# %%time
# off = 0

# groups = (all_male_dist_twice
#             .assign(off=off, # keep offset
#                     roll_win = lambda df: (off + df.start) // window_size) # label for rolling 500kb window
#             .groupby(['indiv_1', 'roll_win', 'off'])
#             )

# old_df = pandas.concat([new_call_rolling_windows(group, args.pwdist_cutoff, MIN_SWEEP_CLADE_SIZE) for name, group in groups])

In [21]:
# new_df.equals(old_df)

In [22]:
# old_df.head()

In [23]:
# #new_df.sort_values(by=['indiv_1', 'indiv_2', 'start']).head()
# new_df.head()

In [24]:
#### NEW #################

lst = list()
# loop over five offsets of 500kb windows
for off in offsets:
    groups = (all_male_dist_twice
                .assign(off=off, # keep offset
                        roll_win = lambda df: (off + df.start) // window_size) # label for rolling 500kb window
#                 .groupby(['indiv_1', 'roll_win', 'off'])
                .groupby(['indiv_1', 'roll_win', 'off'], as_index=False)
                )
    # with Pool(nr_cpu) as p:
    #     df = pandas.concat(p.map(call_rolling_windows, [group for name, group in groups]))
    ##### added pwdist arg to function call
                
#     df = pandas.concat([call_rolling_windows(group, args.pwdist_cutoff, MIN_SWEEP_CLADE_SIZE) for name, group in groups])
#     lst.append(df)

    lst.append(groups.apply(call_rolling_windows, args.pwdist_cutoff, MIN_SWEEP_CLADE_SIZE))


#### NEW #################
    
    
    print('offset:', off)
    
print('Did rolling windows', process.memory_info().rss / 1024**3)

offset: 0
offset: 100000
offset: 200000
offset: 300000
offset: 400000
Did rolling windows 25.583786010742188


In [25]:
process.memory_info().rss / 1024**3

25.58379364013672

In [26]:
# df = pandas.concat(lst)

In [27]:
# df.to_hdf('df.hdf', 'df', format='table', mode='w')
# process.memory_info().rss / 1024**3

In [28]:
# [x.memory_usage(deep=True).sum() for x in lst]

In [29]:
# del lst[:]
# gc.collect()
# process.memory_info().rss / 1024**3

In [30]:
#df = pandas.read_hdf('df.hdf')

In [31]:
# process.memory_info().rss / 1024**3

In [32]:
# groups = df.groupby(['indiv_1', 'start'])

# group = groups.get_group(('B_Han-3', 50000000))

In [33]:
# groups.ngroups

In [34]:
# process.memory_info().rss / 1024**3

In [35]:
# %%timeit
# call_swept(group)

In [36]:
# %%time
# sweep_calls = (pandas.concat(lst)
#                .groupby(['indiv_1', 'start'])
#                 .apply(call_swept)
#                 .reset_index(level=['indiv_1', 'start'])
#                 )

# process.memory_info().rss / 1024**3

In [37]:
# sweep_calls.to_hdf('tmp.hdf', 'df', format='table', mode='w')
# process.memory_info().rss / 1024**3

In [38]:
# concatenate data frames for each offset and call windows as swept
sweep_calls = (pandas.concat(lst)
                .groupby(['indiv_1', 'start'])
                .apply(call_swept)
                .reset_index(level=['indiv_1', 'start'])
                )

print('before', process.memory_info().rss / 1024**3)
#### NEW #################
del lst[:]
gc.collect()
#### NEW #################
print('after', process.memory_info().rss / 1024**3)

print('Did sweep calls')

before 25.510910034179688
after 9.854888916015625
Did sweep calls


In [39]:
%%time
##### partial version of window_stats
import functools
window_stats = functools.partial(_window_stats, pwdist_cutoff=args.pwdist_cutoff, min_sweep_clade_size=MIN_SWEEP_CLADE_SIZE)

# merge window sweep info with distance data
if 'dist_af' in all_male_dist_twice.columns:
    # this is not a simulation
    gr_cols = ['indiv_1', 'start', 'end', 'pop_1', 'region_label_1', 'region_id_1', 'region_1']
else:
    gr_cols = ['indiv_1', 'start', 'end']
df = (all_male_dist_twice
        .groupby(gr_cols)
        .apply(window_stats)
        .reset_index(level=gr_cols)
        )
sweep_data = df.merge(sweep_calls, on=['indiv_1', 'start'])

print('before', process.memory_info().rss / 1024**3)
#### NEW #################
del df
del all_male_dist_twice
gc.collect()
#### NEW #################
print('after', process.memory_info().rss / 1024**3)
print('Did window stats')

before 9.861331939697266
after 8.914207458496094
Did window stats
CPU times: user 11min 57s, sys: 10.2 s, total: 12min 7s
Wall time: 12min 11s


In [40]:
sweep_data.to_hdf('tmp2.hdf', 'df', format='table', mode='w')

In [41]:
%%time

# get run length of swept windows and call windows as part of sweeps (ECH haplotypes)
def run_id(sr):
    return (sr != sr.shift()).cumsum()

sweep_data['run_id'] = (sweep_data
                        .groupby('indiv_1')['called']
                        .apply(run_id)
                       )
sweep_data['run_length'] = (sweep_data
                            .groupby(['indiv_1', 'run_id'])['run_id']
                            .transform(numpy.size)
                           )
sweep_data['swept'] = numpy.bitwise_and(sweep_data['called'], 
                                        sweep_data['run_length'] >= analysis_globals.min_run_length)

if 'called_af' in sweep_data.columns:
    # this is not a simulation
    sweep_data['run_id_af'] = (sweep_data
                            .groupby('indiv_1')['called_af']
                            .apply(run_id)
                        )
    sweep_data['run_length_af'] = (sweep_data
                                .groupby(['indiv_1', 'run_id_af'])['run_id_af']
                                .transform(numpy.size)
                            )
    sweep_data['swept_af'] = numpy.bitwise_and(sweep_data['called_af'], 
                                            sweep_data['run_length_af'] >= analysis_globals.min_run_length)

print('before', process.memory_info().rss / 1024**3)
gc.collect()
print('after', process.memory_info().rss / 1024**3)

print('Did run_length and labeling of swept')    

before 8.807376861572266
after 8.807132720947266
Did run_length and labeling of swept
CPU times: user 6.22 s, sys: 51.6 ms, total: 6.27 s
Wall time: 6.32 s


In [42]:
sweep_data.to_hdf('tmp3.hdf', 'df', format='table', mode='w')

In [43]:


# if args.dump_dist_twice:
#     all_male_dist_twice.to_hdf(args.dump_dist_twice, 'df', 
#                                data_columns=['start', 'end', 
#                                          'indiv_1', 'indiv_2', 
#                                          'pop_1', 'pop_2', 
#                                          'region_label_1', 'region_label_2'],
#                                format='table', mode='w')

    
# write to hdf output file
print('before', process.memory_info().rss / 1024**3)
sweep_data.to_hdf('out.hdf', 'df', format='table', mode='w')
print('after', process.memory_info().rss / 1024**3)

before 8.806892395019531
after 8.806892395019531
